In [ ]:
pip install -q -U tensorflow_addons

In [ ]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
import ipyplot as plt2
import seaborn as sns
from tensorflow.keras.datasets import mnist
import pandas as pd

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255, x_test/255
rad2deg = 180/np.pi

train_morpho = pd.read_csv("train-morpho.csv")[:50000]
test_morpho = pd.read_csv("t10k-morpho.csv")
train_morpho['slant'] = train_morpho['slant'].map(lambda x: x*rad2deg)
test_morpho['slant'] = test_morpho['slant'].map(lambda x: x*rad2deg)


# EDA

In [ ]:
display(train_morpho)

In [ ]:
plt2.plot_class_tabs(x_train, y_train, img_width=28, show_url=False, zoom_scale=3)

In [ ]:
plt.figure()
plt.hist(y_train, rwidth=0.8)
plt.show()

In [ ]:
fig, axes = plt.subplots(5, 2, )
fig.set_figwidth(10)
fig.set_figheight(10)
fig.set_tight_layout(True)
for i in range(len(axes)):
    for j in range(len(axes[i])):
        axes[i, j].hist(train_morpho['slant'][y_train==(2*i+j)])
        axes[i,j].set_title((2*i+j))
        
plt.show()

In [ ]:
class_mean_x = [np.mean(x_train[y_train==i], (0)) for i in range(10)]
plt2.plot_images(class_mean_x)

In [ ]:
fig, axes = plt.subplots(5, 2, )
fig.set_figwidth(10)
fig.set_figheight(15)
fig.set_tight_layout(True)
for i in range(len(axes)):
    for j in range(len(axes[i])):
        slants = train_morpho["slant"][y_train==(2*i+j)]
        axes[i, j].violinplot(slants)
        axes[i,j].set_title(f"{2*i+j} mean: {np.round(np.mean(slants), 4)}")
        axes[i,j].set_ylim(bottom=-1.1, top=1.1)
        
plt.show()

Muchos de los numeros tienen distribuciones muy similares en el slant, ademas hay bastante dispersion.

In [ ]:
plt.figure()
corr_matrix = train_morpho.corr()
sns.heatmap(corr_matrix, annot=True)
plt.show()

La mayor correlacion con el slant la comparte el width. De todas formas es relativamente baja

# Model training

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
from helper import *
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report, ConfusionMatrixDisplay, confusion_matrix
from tensorflow.keras import optimizers as opt
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, Embedding, Concatenate, Input, Normalization
from tensorflow.keras import Sequential, Model
from tensorflow_addons.optimizers import CyclicalLearningRate, Triangular2CyclicalLearningRate, TriangularCyclicalLearningRate


metrics=['mean_squared_error']

#keras.utils.set_random_seed(hash("Los buenos caballos y los buenos estudiantes siempre terminan su carrera.") % 2**32)


width_mean = train_morpho["width"].mean()
width_std = train_morpho["width"].std()
if np.abs(width_mean) > 1:
    train_morpho["width"] = train_morpho["width"].apply(lambda x: (x-width_mean)/width_std)
    print(train_morpho["width"])

In [ ]:
model = Sequential()
model.add(Flatten(input_shape=(28, 28)))
model.add(Dense(100, activation='sigmoid'))
model.add(Dense(100, activation='sigmoid'))
model.add(Dense(1))

clr = Triangular2CyclicalLearningRate(5e-5, 0.01, 5 * 196)

model.compile(optimizer=opt.Adamax(clr), loss='MAE')
model.summary()

In [ ]:
earlyStopCB = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
LearningSchedulerCB = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, min_lr=5e-9, patience=5)

history = model.fit(x=x_train, y=train_morpho["slant"], batch_size=256, epochs=1000, validation_data=(x_test, test_morpho["slant"]), callbacks=[earlyStopCB, LearningSchedulerCB])

In [ ]:
input_img = Input((28, 28))
input_width = Input(1,)
flatten = Flatten()(input_img)
concat = Concatenate()([flatten, input_width])
dense = Dense(100, activation='sigmoid')(concat)
dense = Dense(100, activation='sigmoid')(dense)

out = Dense(1)(dense)
    
model = Model([input_img, input_width], out)

clr = Triangular2CyclicalLearningRate(5e-6, 0.01, 5 * 196)

model.compile(optimizer=opt.Adamax(clr), loss='mae')
model.summary()

In [ ]:
earlyStopCB = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
LearningSchedulerCB = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, min_lr=5e-9, patience=5)

history = model.fit(x=[x_train, train_morpho["width"]], y=train_morpho["slant"], batch_size=256, epochs=1000, validation_data=([x_test, test_morpho["width"]], test_morpho["slant"]), callbacks=[earlyStopCB, LearningSchedulerCB])

In [ ]:

input_img = Input((28, 28))
input_lbl = Input(1,)
flatten = Flatten()(input_img)
embed = Embedding(10, 2, input_length=1)(input_lbl)
embed = Flatten()(embed)
concat = Concatenate()([flatten, embed])
dense = Dense(100, activation='sigmoid')(concat)
dense = Dense(100, activation='sigmoid')(dense)

out = Dense(1)(dense)
    
model = Model([input_img, input_lbl], out)

clr = Triangular2CyclicalLearningRate(1e-7, 0.005, 7 * 469)

model.compile(optimizer=opt.Adamax(0.01), loss='mae')
model.summary()

In [ ]:
earlyStopCB = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
LearningSchedulerCB = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, min_lr=5e-9, patience=5)

history = model.fit(x=[x_train, y_train], y=train_morpho["slant"], batch_size=256, epochs=1000, validation_data=([x_test, y_test], test_morpho["slant"]), callbacks=[earlyStopCB, LearningSchedulerCB])

In [ ]:
aver_adentro = Model(input_lbl, embed)
labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
lindo = aver_adentro.predict(labels)
print(lindo)
plt.figure()
x, y = lindo.T
sns.scatterplot(x=x, y=y, style=labels)

plt.show()
aver = model.predict([x_test, y_test/10])
print(aver)
print(test_morpho)